# Download Sea Surface Temperature Data

Notebook này tải xuống dữ liệu nhiệt độ bề mặt nước biển toàn cầu từ Copernicus Climate Data Store (CDS).

**Nguồn dữ liệu:** [Satellite Sea Surface Temperature](https://cds.climate.copernicus.eu/datasets/satellite-sea-surface-temperature)

**Thông tin dữ liệu:**
- Dataset: satellite-sea-surface-temperature
- Processing Level: Level 4 (gridded, gap-free)
- Sensor: Combined product (nhiều vệ tinh)
- Version: 3.0
- Thời gian: 1980-2023, tất cả các tháng

**Yêu cầu:**
- Cài đặt `cdsapi`: `pip install cdsapi`
- Đăng ký tài khoản tại [CDS](https://cds.climate.copernicus.eu/)
- Cấu hình API key trong file `~/.cdsapirc`

## 1. Cài đặt thư viện cần thiết

In [26]:
!pip install cdsapi

## 2. Import thư viện

In [27]:
import cdsapi
import os
from pathlib import Path
from datetime import datetime
import time

## 3. Cấu hình API key

**Hướng dẫn lấy API key:**
1. Đăng nhập vào [CDS](https://cds.climate.copernicus.eu/)
2. Vào trang [API key](https://cds.climate.copernicus.eu/api-how-to)
3. Copy UID và API Key
4. Tạo file `~/.cdsapirc` với nội dung:
```
url: https://cds.climate.copernicus.eu/api
key: {UID}:{API Key}
```

**Lưu ý:** URL đã thay đổi, sử dụng `/api` thay vì `/api/v2`

Hoặc chạy cell dưới đây để cấu hình:

In [28]:
# Uncomment và điền thông tin API key của bạn
# import os
# os.makedirs(os.path.expanduser('~'), exist_ok=True)
# with open(os.path.expanduser('~/.cdsapirc'), 'w') as f:
#     f.write('url: https://cds.climate.copernicus.eu/api\n')
#     f.write('key: YOUR_UID:YOUR_API_KEY\n')
# print("API key đã được cấu hình!")

## 4. Thiết lập đường dẫn lưu dữ liệu

In [29]:
# Đường dẫn thư mục lưu dữ liệu
data_dir = Path("../../data/raw/sea_surface_temperature")
data_dir.mkdir(parents=True, exist_ok=True)

print(f"Dữ liệu sẽ được lưu tại: {data_dir}")

Dữ liệu sẽ được lưu tại: ../../data/raw/sea_surface_temperature


## 5. Tải dữ liệu từ CDS API (theo từng năm)

**Lưu ý:** 
- Do giới hạn kích thước request, chúng ta sẽ tải dữ liệu theo từng năm
- Mỗi năm sẽ được lưu thành 1 file riêng
- Quá trình này sẽ mất nhiều thời gian
- Có thể merge các file sau bằng xarray

In [30]:
# Khởi tạo CDS API client
try:
    client = cdsapi.Client()
    print("✓ Kết nối CDS API thành công!")
except Exception as e:
    print(f"✗ Lỗi kết nối CDS API: {e}")
    print("Vui lòng kiểm tra lại file ~/.cdsapirc")
    raise

# Định nghĩa dataset
dataset = "satellite-sea-surface-temperature"

# Danh sách các năm cần tải (bỏ qua 1979 vì có thể không có dữ liệu đầy đủ)
years = list(range(1980, 2024))  # 1980-2023

print(f"Bắt đầu tải dữ liệu lúc: {datetime.now()}")
print(f"Tổng số năm cần tải: {len(years)}")
print(f"Dataset: {dataset}")
print("=" * 60)

# Tải dữ liệu theo từng năm
success_count = 0
failed_years = []

for year in years:
    year_str = str(year)
    output_file = data_dir / f"sst_{year}.nc"
    
    # Bỏ qua nếu file đã tồn tại
    if output_file.exists():
        print(f"✓ Năm {year}: File đã tồn tại, bỏ qua")
        success_count += 1
        continue
    
    # Request cho 1 năm, chỉ lấy ngày 15 mỗi tháng
    request = {
        "processinglevel": "level_4",
        "sensor_on_satellite": "combined_product",
        "version": "3_0",
        "year": [year_str],
        "month": ["01", "02", "03", "04", "05", "06", 
                  "07", "08", "09", "10", "11", "12"],
        "day": ["15"],  # Chỉ lấy giữa tháng
        "variable": "all"
    }
    
    print(f"\n[{success_count + 1}/{len(years)}] Đang tải năm {year}...")
    
    try:
        # Tải dữ liệu
        result = client.retrieve(dataset, request)
        result.download(str(output_file))
        
        # Kiểm tra file size
        file_size_mb = os.path.getsize(output_file) / (1024 * 1024)
        print(f"  ✓ Thành công! Kích thước: {file_size_mb:.2f} MB")
        success_count += 1
        
        # Đợi 2 giây giữa các request để tránh quá tải server
        if year != years[-1]:
            time.sleep(2)
            
    except Exception as e:
        print(f"  ✗ Lỗi: {e}")
        failed_years.append(year)
        continue

# Tổng kết
print("\n" + "=" * 60)
print("KẾT QUẢ TỔNG KẾT:")
print(f"✓ Tải thành công: {success_count}/{len(years)} năm")
if failed_years:
    print(f"✗ Tải thất bại: {len(failed_years)} năm: {failed_years}")
print(f"Hoàn thành lúc: {datetime.now()}")
print(f"Dữ liệu được lưu tại: {data_dir}")

✓ Kết nối CDS API thành công!
Bắt đầu tải dữ liệu lúc: 2026-01-15 02:29:51.359399
Tổng số năm cần tải: 43
Dataset: satellite-sea-surface-temperature

[1/43] Đang tải năm 1981...


2026-01-15 02:29:52,240 WARNING [2026-01-14T00:00:00] We are currently experiencing data access issues that may cause some download requests to fail. We are working to resolve this issue. Our team is actively investigating and working to resolve the issue as quickly as possible.
2026-01-15 02:29:52,241 INFO Request ID is 08f65aa6-a2c5-4d6c-9ee0-f5f1182e30d2
2026-01-15 02:29:52,519 INFO status has been updated to accepted
2026-01-15 02:30:07,218 INFO status has been updated to running
2026-01-15 02:30:45,632 INFO status has been updated to successful


  ✓ Thành công! Kích thước: 182.51 MB

[2/43] Đang tải năm 1982...


2026-01-15 02:31:54,941 WARNING [2026-01-14T00:00:00] We are currently experiencing data access issues that may cause some download requests to fail. We are working to resolve this issue. Our team is actively investigating and working to resolve the issue as quickly as possible.
2026-01-15 02:31:54,941 INFO Request ID is 7654ba4d-3c30-4af6-9a52-05ba6379ddb0
2026-01-15 02:31:55,263 INFO status has been updated to accepted
2026-01-15 02:32:09,967 INFO status has been updated to running
2026-01-15 02:32:29,568 INFO status has been updated to successful


  ✓ Thành công! Kích thước: 186.85 MB

[3/43] Đang tải năm 1983...


...

  ✓ Thành công! Kích thước: 187.39 MB

[42/43] Đang tải năm 2022...


2026-01-15 03:49:25,942 WARNING [2026-01-14T00:00:00] We are currently experiencing data access issues that may cause some download requests to fail. We are working to resolve this issue. Our team is actively investigating and working to resolve the issue as quickly as possible.
2026-01-15 03:49:25,944 INFO Request ID is 08d48755-ff50-4b35-9951-ecc70be7d3d4
2026-01-15 03:49:26,210 INFO status has been updated to accepted
2026-01-15 03:49:40,868 INFO status has been updated to running
2026-01-15 03:50:17,757 INFO status has been updated to successful


  ✓ Thành công! Kích thước: 186.78 MB

[43/43] Đang tải năm 2023...


2026-01-15 03:51:16,741 WARNING [2026-01-14T00:00:00] We are currently experiencing data access issues that may cause some download requests to fail. We are working to resolve this issue. Our team is actively investigating and working to resolve the issue as quickly as possible.
2026-01-15 03:51:16,742 INFO Request ID is fd918bfb-5f90-4a99-9622-cdc6c0bb73e3
2026-01-15 03:51:17,004 INFO status has been updated to accepted
2026-01-15 03:51:39,881 INFO status has been updated to running
2026-01-15 03:52:34,973 INFO status has been updated to successful
                                                                                         

  ✓ Thành công! Kích thước: 187.10 MB

KẾT QUẢ TỔNG KẾT:
✓ Tải thành công: 43/43 năm
Hoàn thành lúc: 2026-01-15 03:53:35.621046
Dữ liệu được lưu tại: ../../data/raw/sea_surface_temperature


## 6. Kiểm tra dữ liệu đã tải

In [31]:
import os

# Liệt kê tất cả các file đã tải
files = sorted(data_dir.glob("sst_*.nc"))

if files:
    total_size = sum(os.path.getsize(f) for f in files)
    total_size_mb = total_size / (1024 * 1024)
    
    print(f"✓ Đã tải {len(files)} file thành công!")
    print(f"  - Tổng kích thước: {total_size_mb:.2f} MB")
    print(f"  - Thư mục: {data_dir}")
    print("\nDanh sách file:")
    for f in files:
        size_mb = os.path.getsize(f) / (1024 * 1024)
        print(f"  - {f.name}: {size_mb:.2f} MB")
else:
    print("✗ Chưa có file nào được tải xuống.")

✓ Đã tải 43 file thành công!
  - Tổng kích thước: 7925.31 MB
  - Thư mục: ../../data/raw/sea_surface_temperature

Danh sách file:
  - sst_1981.nc: 182.51 MB
  - sst_1982.nc: 186.85 MB
  - sst_1983.nc: 184.76 MB
  - sst_1984.nc: 187.51 MB
  - sst_1985.nc: 190.85 MB
  - sst_1986.nc: 189.90 MB
  - sst_1987.nc: 186.95 MB
  - sst_1988.nc: 186.58 MB
  - sst_1989.nc: 188.83 MB
  - sst_1990.nc: 186.97 MB
  - sst_1991.nc: 187.21 MB
  - sst_1992.nc: 190.07 MB
  - sst_1993.nc: 190.88 MB
  - sst_1994.nc: 191.25 MB
  - sst_1995.nc: 193.56 MB
  - sst_1996.nc: 186.47 MB
  - sst_1997.nc: 183.46 MB
  - sst_1998.nc: 183.35 MB
  - sst_1999.nc: 183.30 MB
  - sst_2000.nc: 183.24 MB
  - sst_2001.nc: 183.38 MB
  - sst_2002.nc: 181.69 MB
  - sst_2003.nc: 181.24 MB
  - sst_2004.nc: 181.14 MB
  - sst_2005.nc: 181.20 MB
  - sst_2006.nc: 181.33 MB
  - sst_2007.nc: 181.57 MB
  - sst_2008.nc: 180.83 MB
  - sst_2009.nc: 180.63 MB
  - sst_2010.nc: 181.26 MB
  - sst_2011.nc: 180.57 MB
  - sst_2012.nc: 179.90 MB
  - ss

## 7. Merge các file thành 1 file duy nhất (Optional)

Sử dụng xarray để merge tất cả các file theo thứ tự thời gian:

In [32]:
# Uncomment để cài đặt
# !pip install xarray netCDF4

In [33]:
# Uncomment để merge các file
# import xarray as xr
# 
# print("Đang merge các file...")
# files = sorted(data_dir.glob("sst_*.nc"))
# 
# # Mở và merge tất cả các file
# datasets = [xr.open_dataset(f) for f in files]
# merged_ds = xr.concat(datasets, dim='time')
# 
# # Lưu file merged
# output_merged = data_dir.parent / "sea_surface_temperature_1981-2023_merged.nc"
# merged_ds.to_netcdf(output_merged)
# 
# # Đóng các dataset
# for ds in datasets:
#     ds.close()
# merged_ds.close()
# 
# print(f"✓ Đã merge thành công!")
# print(f"  - File output: {output_merged}")
# print(f"  - Kích thước: {os.path.getsize(output_merged) / (1024**2):.2f} MB")

## 8. Xem thông tin dữ liệu (Optional)

In [34]:
# Uncomment để xem thông tin dữ liệu của 1 năm bất kỳ
# import xarray as xr
# 
# # Mở file của năm đầu tiên
# files = sorted(data_dir.glob("sst_*.nc"))
# if files:
#     ds = xr.open_dataset(files[0])
#     
#     print("Thông tin dataset mẫu (năm đầu tiên):")
#     print(ds)
#     
#     print("\nCác biến trong dataset:")
#     for var in ds.data_vars:
#         print(f"  - {var}: {ds[var].attrs.get('long_name', 'N/A')}")
#     
#     ds.close()

## Kết thúc

Dữ liệu nhiệt độ bề mặt nước biển đã được tải xuống thành công theo từng năm!

**Ưu điểm của phương pháp này:**
- Tránh lỗi "request too large"
- Có thể tiếp tục tải nếu bị gián đoạn
- Dễ dàng quản lý và kiểm tra từng năm

**Các bước tiếp theo:**
1. Merge các file thành 1 file (nếu cần)
2. Xử lý và làm sạch dữ liệu
3. Phân tích xu hướng nhiệt độ theo thời gian
4. Tích hợp với dữ liệu mực nước biển